<a href="https://colab.research.google.com/github/kys8411/AIFFEL_Exploration/blob/main/%5B%EC%9D%B4%EA%B8%B0%EC%B0%BD_%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC%5D_5_3_%ED%95%99%EC%8A%B5_%EB%A7%88%EC%B9%9C_%EB%AA%A8%EB%8D%B8%EC%9D%84_%EC%8B%A4%EC%A0%84_%ED%88%AC%EC%9E%85%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패키지 설치
pip 명령어로 의존성 있는 패키지를 설치합니다.



In [ ]:
# !pip install ratsnlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 kB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.8 MB/s eta 0:00:00


# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [ ]:
## 이 코드는 분류 작업의 배포 설정을 정의하기 위해 ratsnlp 라이브러리의 ClassificationDeployArguments 클래스를 사용합니다.

from ratsnlp.nlpbook.classification import ClassificationDeployArguments
# 모듈 임포트: 분류 배포를 위한 설정 클래스를 가져옴.

args = ClassificationDeployArguments(
    # ClassificationDeployArguments 클래스의 생성자를 호출하여 배포 설정을 초기화합니다.
    pretrained_model_name="beomi/kcbert-base",
    # 사용할 사전 학습된 모델의 이름입니다. 여기서는 "beomi/kcbert-base"를 사용합니다.
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-paircls",
    # 사전 학습된 모델을 기반으로 훈련된 분류 모델의 체크포인트가 저장된 경로입니다.
    max_seq_length=64,
    # 입력 시퀀스의 최대 길이입니다. 긴 문장은 이 길이에 맞게 잘릴 것이며, 짧은 문장은 패딩됩니다.
)

## 결과적으로, args 변수에는 배포를 위한 설정이 저장됩니다. 이 설정은 후속 단계에서 분류 모델을 배포하거나 사용할 때 필요합니다.

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-paircls/epoch=1-val_loss=0.82-v1.ckpt


# 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어 들입니다.

In [ ]:
## 이 코드는 훌련된 체크포인트를 불러와서 BERT 모델을 배포 준비 상태로 만듭니다.

import torch
from transformers import BertConfig, BertForSequenceClassification

## 체크 포인트 불러오기
fine_tuned_model_ckpt = torch.load(
    # 훈련된 모델의 체크포인트를 불러옵니다. 이 체크포인트에는 모델의 가중치와 바이어스 등의 파라미터가 포함되어 있습니다.
    args.downstream_model_checkpoint_fpath,
    # 훈련된 모델의 체크포인트 파일 경로입니다.
    map_location=torch.device("cpu")
    # 체크포인트를 CPU로 불러옵니다.
)

## 모델 설정 불러오기
pretrained_model_config = BertConfig.from_pretrained(
    #  BERT 모델의 기본 설정을 불러옵니다.
    args.pretrained_model_name,
    # 사용할 사전 학습된 모델의 이름입니다.
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
    # 분류 작업의 레이블 수를 체크포인트에서 불러옵니다.
)

## 모델 초기화
model = BertForSequenceClassification(pretrained_model_config)
# BERT 분류 모델을 초기화 합니다.

## 모델 가중치 설정
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
# 훈련된 체크포인트에서 불러온 모델의 가중치와 바이어스를 BERT 분류 모델에 적용합니다.
# k.replace("model.", ""): 체크포인트의 키 이름을 조정하여 BERT 모델의 키 이름과 일치시킵니다.

## 모델 평가 모드 설정
model.eval()
# model.eval(): 모델을 평가 모드로 설정합니다. 이렇게 하면 드롭아웃 및 배치 정규화와 같은 학습 전용 기능들이 비활성화됩니다.


## 결과적으로, 이 코드를 실행하면 훈련된 체크포인트를 기반으로 BERT 분류 모델을 배포 준비 상태로 만듭니다.
## 이 모델은 이후 실제 데이터에 대한 예측을 수행할 준비가 되었습니다.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
## 이 코드는 transformers 라이브러리를 사용하여 BERT 토크나이저를 초기화합니다.

from transformers import BertTokenizer
# 모듈 임포트: 라이브러리에서 BERT 모델에 맞는 토크나이저를 가져옵니다.

tokenizer = BertTokenizer.from_pretrained(
    # 토크나이저 초기화: 사전 학습된 모델의 이름을 기반으로 해당 모델에 맞는 토크나이저를 불러옵니다.
    args.pretrained_model_name,
    # 사용할 사전 학습된 모델의 이름입니다. 여기서는 "beomi/kcbert-base"를 사용합니다.
    do_lower_case=False,
    # 이 옵션은 입력 텍스트를 소문자로 변환할 것인지를 결정합니다. False로 설정되어 있으므로 소문자로 변환하지 않습니다.
)

## tokenizer는 주어진 텍스트를 BERT 모델에 입력할 수 있는 형태로 토큰화하는 데 사용됩니다.

# 인퍼런스 함수 선언
인퍼런스 함수를 선언합니다.

In [ ]:
## 이 함수는 주장(premise)과 가설(hypothesis)을 입력받아, 두 문장 간의 관계를 분류하는 추론(inference)를 수행합니다.
## 토큰화: tokenizer를 사용해 주장과 가설을 BERT 모델의 입력 형식에 맞게 변환합니다. 입력은 최대 길이(max_seq_length)로 제한되며, 길게 초과할 경우 잘라내고 짧을 경우 패딩됩니다.
## 모델 추론: 주어진 입력을 모델에 넣어 결과를 얻습니다. torch.no_grad()는 그라디언트 계산을 방지하여 메모리 사용을 최소화하며 속도를 향상시킵니다.
## 결과 분석: 모델의 출력 logits는 softmax 함수를 통해 확률로 변환됩니다. 그리고 각 관계에 대한 확률과 예측된 관계를 결정합니다.
## 결과 반환: 함수는 관계의 예측, 각 관계에 대한 확률 및 그 확률을 표현하는 문자열 포맷을 포함하는 딕셔너리를 반환합니다.
## 결과적으로, 이 함수는 두 문장이 주어지면 두 문장 간의 관계(참, 거짓, 중립)를 분류하고 그 결과를 반환합니다.


def inference_fn(premise, hypothesis):
    # 함수 inference_fn는 두 개의 인자, premise(주장)과 hypothesis(가설)를 받습니다.
    inputs = tokenizer(
        [(premise, hypothesis)],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    # 주장과 가설을 튜플로 묶어 토크나이저에 전달하여 BERT 입력 형식으로 변환합니다. 길이가 max_seq_length를 초과하면 잘라내고, 그보다 짧으면 패딩으로 채웁니다.


    with torch.no_grad():
        # 이 코드 블록 내에서는 모델 추론 시 그라디언트(미분값) 계산을 하지 않도록 합니다. 이렇게 하면 메모리를 덜 사용하고 속도도 빨라집니다.

        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        # 변환된 입력을 모델에 넣어 결과(outputs)를 얻습니다.

        prob = outputs.logits.softmax(dim=1)
        # 모델의 출력값을 확률로 변환합니다.

        entailment_prob = round(prob[0][0].item(), 2)
        contradiction_prob = round(prob[0][1].item(), 2)
        neutral_prob = round(prob[0][2].item(), 2)
        # 각 관계에 대한 확률을 소수 둘째자리까지 반올림합니다.

        if torch.argmax(prob) == 0:
            pred = "참 (entailment)"
        elif torch.argmax(prob) == 1:
            pred = "거짓 (contradiction)"
        else:
            pred = "중립 (neutral)"
        # 가장 높은 확률을 가진 관계를 선택하여 예측합니다.

    return {
        'premise': premise,
        'hypothesis': hypothesis,
        'prediction': pred,
        'entailment_data': f"참 {entailment_prob}",
        'contradiction_data': f"거짓 {contradiction_prob}",
        'neutral_data': f"중립 {neutral_prob}",
        'entailment_width': f"{entailment_prob * 100}%",
        'contradiction_width': f"{contradiction_prob * 100}%",
        'neutral_width': f"{neutral_prob * 100}%",
    }
    # 함수는 주장, 가설, 예측된 관계, 각 관계에 대한 확률 및 그 확률을 표현하는 문자열 포맷을 포함하는 딕셔너리를 반환합니다.
    # 간단히 말하면, 이 함수는 두 문장을 입력받아 두 문장 사이의 관계를 예측하고, 그 관계와 관련된 확률값들을 반환합니다.

# 웹서비스 만들기 준비

`ngrok`은 코랩 로컬에서 실행 중인 웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구입니다. `ngrok`을 실행하려면 [회원가입](https://dashboard.ngrok.com/signup) 후 [로그인](https://dashboard.ngrok.com/login)을 한 뒤 [이곳](https://dashboard.ngrok.com/get-started/your-authtoken)에 접속해 인증 토큰(authtoken)을 확인해야 합니다. 예를 들어 확인된 `authtoken`이 `test111`이라면 다음과 같이 실행합니다.

```bash
!mkdir /root/.ngrok2 && echo "authtoken: test111" > /root/.ngrok2/ngrok.yml
```

In [ ]:
!mkdir /root/.ngrok2 && echo "authtoken: {이곳에 확인된 인증 토큰을 입력하세요}" > /root/.ngrok2/ngrok.yml

# 웹서비스 개시
아래처럼 실행해 인퍼런스 함수를 웹서비스로 만듭니다.

In [ ]:
from ratsnlp.nlpbook.paircls import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app 'ratsnlp.nlpbook.paircls.deploy'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 790, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll